# Part 1: Data Extraction


In [1]:
pip install selenium pandas webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchWindowException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
CATEGORIES = {
    "Mouse": "https://www.banggood.com/Wholesale-Mouse-ca-5033.html",
    "Monitor": "https://www.banggood.com/Wholesale-Monitor-ca-5016.html",
    "Bags": "https://www.banggood.com/Wholesale-Bags-and-Backpacks-ca-6031.html",
    "Juicers": "https://www.banggood.com/Wholesale-Juicers-ca-9067.html",
    "chargers": "https://www.banggood.com/Wholesale-Chargers-ca-1055.html"
}

PAGES_TO_SCRAPE = 3

def setup_driver():
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
    
    options.page_load_strategy = 'eager' 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(25)
    return driver

def extract_data(driver, category_name):
    products_data = []
    
    cards = []
    possible_containers = ["li.p-item", "div.p-wrap", "div.list-item", "li.gl-item", "div.product-item"]
    
    for container in possible_containers:
        found = driver.find_elements(By.CSS_SELECTOR, container)
        if len(found) > 0:
            cards = found
            break 
    
    if not cards:
        print(f"    > Warning: No product cards found.")
        return []

    for card in cards:
        name = "N/A"
        price = "N/A"
        url = "N/A"
        rating = "N/A"
        reviews = "0 reviews"

        try:
            name = card.find_element(By.CSS_SELECTOR, "a.title").text.strip()
        except:
            try:
                name = card.find_element(By.CSS_SELECTOR, "a.p-title").text.strip()
            except:
                try:
                    name = card.find_element(By.CSS_SELECTOR, ".title a").text.strip()
                except:
                    pass
        try:
            price = card.find_element(By.CSS_SELECTOR, "span.price-box").text.strip()
        except:
            try:
                price = card.find_element(By.CSS_SELECTOR, ".price").text.strip()
            except:
                try:
                    price = card.find_element(By.CSS_SELECTOR, ".product-price").text.strip()
                except:
                    pass

        try:
            url = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        except:
            pass

        try:
            reviews = card.find_element(By.CSS_SELECTOR, "a.review").text.strip()
        except:
            pass
            
        try:
            rating = card.find_element(By.CSS_SELECTOR, "span.star").get_attribute("data-score")
        except:
            pass
            
        if name == "N/A" and price == "N/A":
            continue

        products_data.append({
            "Category": category_name,
            "Name": name,
            "Price": price,
            "Rating": rating,
            "Reviews": reviews,
            "URL": url
        })

    return products_data

def main():
    driver = setup_driver()
    all_products = []

    print("Starting Scraper (Fixed Selectors)...")
    print("-------------------------------------")

    for cat_name, base_url in CATEGORIES.items():
        print(f"Processing Category: {cat_name}")
        
        for page in range(1, PAGES_TO_SCRAPE + 1):
            if page == 1:
                target_url = base_url
            else:
                target_url = f"{base_url}?page={page}"

            print(f"  - Scraping Page {page}...", end=" ")
            
            try:
                try:
                    driver.get(target_url)
                except TimeoutException:
                    driver.execute_script("window.stop();")

                driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 3);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 1.5);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                products = extract_data(driver, cat_name)
                all_products.extend(products)
                print(f"-> Found {len(products)} items.")

            except (NoSuchWindowException, WebDriverException):
                print(f"\n    !!! Browser Error. Restarting driver...")
                try: driver.quit()
                except: pass
                driver = setup_driver()
                continue

            time.sleep(2) 

    driver.quit()

    print("-------------------")
    if all_products:
        df = pd.DataFrame(all_products)
        # Save to CSV
        df.to_csv("banggood_products.csv", index=False)
        print(f"Success! Scraped {len(all_products)} items.")
        print("Data saved to 'banggood_products.csv'")
        print(df.head()) # Show first few lines
    else:
        print("No data scraped.")

if __name__ == "__main__":
    main()

Starting Scraper (Fixed Selectors)...
-------------------------------------
Processing Category: Mouse
-> Found 60 items.1... 
  - Scraping Page 2... -> Found 23 items.
  - Scraping Page 3...     > Warning: No product cards found.
-> Found 0 items.
Processing Category: Monitor
-> Found 8 items. 1... 
  - Scraping Page 2...     > Warning: No product cards found.
-> Found 0 items.
  - Scraping Page 3...     > Warning: No product cards found.
-> Found 0 items.
Processing Category: Bags
-> Found 60 items.1... 
  - Scraping Page 2... -> Found 8 items.
  - Scraping Page 3...     > Warning: No product cards found.
-> Found 0 items.
Processing Category: Juicers
-> Found 6 items. 1... 
  - Scraping Page 2...     > Warning: No product cards found.
-> Found 0 items.
  - Scraping Page 3...     > Warning: No product cards found.
-> Found 0 items.
Processing Category: chargers
-> Found 60 items.1... 
  - Scraping Page 2... -> Found 60 items.
  - Scraping Page 3... -> Found 38 items.
----------------

# Part 2: Data Cleaning & Transformation

In [57]:
import pandas as pd

df = pd.read_csv("banggood_products.csv")
print(df.head())

# Clean Price
df["Price_Clean"] = df["Price"].replace("N/A", None)
df["Price_Clean"] = df["Price_Clean"].str.replace(r"[^\d.]", "", regex=True)
df["Price_Clean"] = df["Price_Clean"].astype(float)
df["Price_Clean"] = df["Price_Clean"].fillna(df["Price_Clean"].median())  # assign back instead of inplace

# Clean Rating
df["Rating_Clean"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Rating_Clean"] = df["Rating_Clean"].fillna(df["Rating_Clean"].median())  # assign back

# Clean Reviews
df["Reviews_Clean"] = df["Reviews"].str.extract(r"(\d+)")[0]  # extract numbers
df["Reviews_Clean"] = pd.to_numeric(df["Reviews_Clean"], errors="coerce").fillna(0).astype(int)

# Derived features
df["Price_per_Review"] = df["Price_Clean"] / (df["Reviews_Clean"] + 1)
df["High_Rating_Flag"] = (df["Rating_Clean"] >= 4).astype(int)
df["Price_Category"] = pd.cut(df["Price_Clean"], bins=[0, 20, 50, 100, 500],
                              labels=["Cheap", "Moderate", "Expensive", "Luxury"])

# Save cleaned CSV
df.to_csv("banggood_products_cleaned.csv", index=False)
print("Cleaned data saved successfully!")
print(df.head())

  Category                                               Name     Price  \
0    Mouse  Xiaomi MIIIW Lite Wireless Mute Mouse 800/1200...  US$11.99   
1    Mouse  HXSJ T90 2.4G Triple Mode Wireless Mouse 800-3...  US$12.99   
2    Mouse  Dual Mode BT3.0/5.2 2.4G Wireless Mouse Adjust...   US$5.99   
3    Mouse  RAPOO VT9PRO Dual Mode Wireless Gaming Mouse P...  US$31.99   
4    Mouse  HXSJ T69 2.4G Wireless Gaming Mouse with RGB L...  US$12.99   

   Rating      Reviews                                                URL  
0     NaN    7 reviews  https://www.banggood.com/Xiaomi-MIIIW-Lite-Wir...  
1     NaN   64 reviews  https://www.banggood.com/HXSJ-T90-2_4G-Triple-...  
2     NaN  331 reviews  https://www.banggood.com/Dual-Mode-BT3_0-or-5_...  
3     NaN    2 reviews  https://www.banggood.com/RAPOO-VT9PRO-Dual-Mod...  
4     NaN   19 reviews  https://www.banggood.com/HXSJ-T69-2_4G-Wireles...  
Cleaned data saved successfully!
  Category                                               Nam

# Part 4: Load Data into SQL Server


In [ ]:
import pyodbc

conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-QQN1OAH;"
    "Database=BanggoodDB;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE BanggoodProducts (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    Category VARCHAR(100),
    Product NVARCHAR(MAX),
    Price_Clean FLOAT,
    Rating_Clean FLOAT,
    Reviews_Clean INT,
    Price_per_Review FLOAT,
    High_Rating_Flag BIT,
    Price_Category VARCHAR(50),
    URL NVARCHAR(MAX)

)
""")
conn.commit()

print("Table Created Successfully!")

In [65]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-KG6F56I;"
    "Database=BanggoodDB;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE BanggoodProducts (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    Category VARCHAR(100),
    Product NVARCHAR(MAX),
    Price_Clean FLOAT,
    Rating_Clean FLOAT,
    Reviews_Clean INT,
    Price_per_Review FLOAT,
    High_Rating_Flag BIT,
    Price_Category VARCHAR(50),
    URL NVARCHAR(MAX)

)
""")
conn.commit()

print("Table Created Successfully!")



Table Created Successfully!


In [70]:
import pandas as pd
import pyodbc


df = pd.read_csv("banggood_products_cleaned.csv")
df.columns = df.columns.str.strip()  # remove any extra spaces

# Rename 'Name' to 'Product' to match SQL table
df = df.rename(columns={"Name": "Product"})

required_columns = [
    "Category", "Product", "Price_Clean", "Rating_Clean", "Reviews_Clean",
    "Price_per_Review", "High_Rating_Flag", "Price_Category", "URL"
]

# Check which columns exist
existing_columns = [col for col in required_columns if col in df.columns]
df = df[existing_columns]


#Convert the numeric columns

numeric_cols = ["Price_Clean", "Rating_Clean", "Reviews_Clean", "Price_per_Review"]

# Remove currency symbols and convert to numeric
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('[\$,US]', '', regex=True)  # remove $ or US symbols
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill missing Rating_Clean with 0 (optional)
if "Rating_Clean" in df.columns:
    df["Rating_Clean"] = df["Rating_Clean"].fillna(0)

# Fill missing Reviews_Clean or Price_per_Review with 0
if "Reviews_Clean" in df.columns:
    df["Reviews_Clean"] = df["Reviews_Clean"].fillna(0)
if "Price_per_Review" in df.columns:
    df["Price_per_Review"] = df["Price_per_Review"].fillna(0)

if "High_Rating_Flag" in df.columns:
    df["High_Rating_Flag"] = df["High_Rating_Flag"].fillna(0).astype(int)



df = df.dropna(subset=["Product", "Price_Clean", "URL"])


# Data Insertion

values_to_insert = df[required_columns].values.tolist() if not df.empty else []

if not values_to_insert:
    print("No valid data found for insertion. Please check your CSV and preprocessing.")
else:

 # Connect to SQL Server

    conn = pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=DESKTOP-KG6F56I;"
        "Database=BanggoodDB;"
        "Trusted_Connection=yes;"
    )
    cursor = conn.cursor()

 
    insert_query = """
        INSERT INTO BanggoodProducts (
            Category, Product, Price_Clean, Rating_Clean, Reviews_Clean,
            Price_per_Review, High_Rating_Flag, Price_Category, URL
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    try:
        cursor.executemany(insert_query, values_to_insert)
        conn.commit()
        print(f"All Data Inserted Successfully! Total rows: {len(values_to_insert)}")
    except Exception as e:
        print("Error inserting data:", e)
    finally:
        cursor.close()
        conn.close()

All Data Inserted Successfully! Total rows: 323
